In [26]:
from os import environ, listdir, path

environ["HF_HOME"] = "/oak/stanford/groups/mrivas/projects/multiomics/tlmenest/LLMChemCreator/huggingface"

In [42]:
from transformers import AutoModel, AutoTokenizer
from torch        import no_grad, save, load
from Bio.PDB      import PDBParser
from Bio.SeqUtils import seq1

In [4]:
model_name = "facebook/esm2_t33_650M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Define your PDB ID and file path
PDB_ID = "YOUR_PDB_ID_HERE" 
PDB_file_path = "proteins/apol1.pdb"
pdbparser = PDBParser()

structure = pdbparser.get_structure(PDB_ID, PDB_file_path)
chains = {chain.id: seq1(''.join(residue.resname.strip() for residue in chain if residue.id[0] == " " and residue.resname.isalpha()), undef_code="X") for chain in structure.get_chains()}

In [15]:
chain = chains['A']

In [7]:
pdb_file_path = "proteins/apol1.pdb"
sequence = extract_sequence_from_pdb(pdb_file_path)

In [16]:
inputs = tokenizer(chain, return_tensors="pt")

with no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state  

In [18]:
embeddings.shape

torch.Size([1, 400, 1280])

In [19]:
embeddings

tensor([[[ 0.0294, -0.0003,  0.0925,  ..., -0.1806,  0.1562, -0.0058],
         [-0.1946,  0.1738, -0.0511,  ...,  0.1141, -0.0597, -0.0010],
         [-0.1579,  0.0912,  0.0042,  ...,  0.2209,  0.1433,  0.0470],
         ...,
         [-0.0617, -0.0068, -0.0116,  ...,  0.1978,  0.0556,  0.0226],
         [ 0.0489,  0.1007,  0.0295,  ...,  0.0370,  0.2020, -0.1112],
         [-0.0444,  0.0979,  0.0806,  ..., -0.1284,  0.2634, -0.0867]]])

In [22]:
protein_files = listdir('proteins')
pdb_files     = [file for file in protein_files if file.endswith('.pdb')]

In [31]:
pdbparser = PDBParser()
chains    = []

for file in pdb_files:

    PDB_file_path = path.join('proteins', file)
    structure = pdbparser.get_structure(PDB_ID, PDB_file_path)
    output    = {chain.id: seq1(''.join(residue.resname.strip() for residue in chain if residue.id[0] == " " and residue.resname.isalpha()), undef_code="X") for chain in structure.get_chains()}

    chain = output['A']
    (chains.append(chain))

In [46]:
for chain in chains:

    print(len(chain))

398
1988
634
1261


In [33]:
%%time

embeddings = []

for chain in chains:

    inputs = tokenizer(chain, return_tensors="pt")

    with no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state  
        print('Done')
    embeddings.append(embedding)

CPU times: user 1min 37s, sys: 27 s, total: 2min 4s
Wall time: 2min 6s


In [34]:
embeddings

[tensor([[[ 0.0294, -0.0003,  0.0925,  ..., -0.1806,  0.1562, -0.0058],
          [-0.1946,  0.1738, -0.0511,  ...,  0.1141, -0.0597, -0.0010],
          [-0.1579,  0.0912,  0.0042,  ...,  0.2209,  0.1433,  0.0470],
          ...,
          [-0.0617, -0.0068, -0.0116,  ...,  0.1978,  0.0556,  0.0226],
          [ 0.0489,  0.1007,  0.0295,  ...,  0.0370,  0.2020, -0.1112],
          [-0.0444,  0.0979,  0.0806,  ..., -0.1284,  0.2634, -0.0867]]]),
 tensor([[[ 0.0599,  0.0072,  0.0975,  ..., -0.2745,  0.1429,  0.0200],
          [ 0.0078, -0.1106, -0.0695,  ...,  0.0252, -0.1169,  0.1243],
          [-0.0485,  0.1942, -0.0411,  ...,  0.0175, -0.0770,  0.1818],
          ...,
          [ 0.0195,  0.1008, -0.1493,  ..., -0.0391, -0.2148, -0.0202],
          [ 0.0948,  0.0608, -0.1663,  ..., -0.0629, -0.1604,  0.1686],
          [ 0.0228, -0.0361,  0.0298,  ..., -0.1657,  0.1408,  0.0105]]]),
 tensor([[[ 0.0334,  0.0339,  0.0485,  ..., -0.3165,  0.1312,  0.0030],
          [-0.0057,  0.0353,

In [35]:
pdb_files[0][:-3]

'apol1.'

In [41]:
for i in range(len(pdb_files)):

    save_path = path.join('proteins/embeddings/', (pdb_files[i][:-4] + '_embedding.pt'))
    save(embeddings[i], save_path)

In [44]:
loaded_tensor = load('proteins/embeddings/apol1_embedding.pt')

# Display the loaded tensor
loaded_tensor.shape

torch.Size([1, 400, 1280])

In [ ]:
def canonicalize(s):
    return Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True, isomericSmiles=False)

def batch_split(data, batch_size=64):
    for i in range(0, len(data), batch_size):
        yield data[i:i+batch_size]

def embed_and_save(model, df, tokenizer, batch_size=64):
    model.eval()
    for batch in batch_split(df, batch_size):
        smiles = batch['canonical_smiles'].apply(canonicalize)
        batch_enc = tokenizer.batch_encode_plus(smiles.tolist(), padding=True, add_special_tokens=True, return_tensors='pt')
        idx, mask = batch_enc['input_ids'], batch_enc['attention_mask']
        with torch.no_grad():
            token_embeddings = model(idx, attention_mask=mask).last_hidden_state
        input_mask_expanded = mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        embeddings = sum_embeddings / sum_mask
        for i, chembl_id in enumerate(batch['chembl_id']):
            # Ensure the directory exists
            os.makedirs('ligands', exist_ok=True)
            # Save each tensor
            torch.save(embeddings[i].cpu(), f'ligands/{chembl_id}.pt')

In [6]:
from os import listdir

In [7]:
embs = listdir('ligands/')

In [9]:
len(embs)

869510